# CS 7646: Machine Learning for Trading
## Week 2 - 08/29/2022
### Lessons: 01-05 to 01-08

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
data_dir = "../data/"

## 01-05 Incomplete data

### Pristine data

What people think:
- Perfect data recorded minute by minute.
- No gaps or missing data points.

Reality:
- Data is an amalgamation.
- Not all stocks trade every day.

### Why data goes missing?

In [ ]:
def symbol_to_path(symbol, base_dir=data_dir):
    """Return CSV file path given ticker symbol."""
    return os.path.join(base_dir, f"{str(symbol)}.csv")


def get_data(symbols, dates):
    """Read stock data (adjusted close) for given symbols from CSV files."""
    df = pd.DataFrame(index=dates)
    if 'SPY' not in symbols:  # add SPY for reference, if absent
        symbols.insert(0, 'SPY')
    
    for symbol in symbols:
        df_temp = pd.read_csv(f"{symbol_to_path(symbol)}",
                              index_col='Date',
                              parse_dates=True,
                              usecols=['Date', 'Adj Close'],
                              na_values=['nan'])
        df_temp = df_temp.rename(columns={'Adj Close': symbol})
        df = df.join(df_temp)  # use default how='left'
        if symbol == 'SPY':
            df = df.dropna(subset=['SPY'])
        
    return df

In [ ]:
def plot_selected(df, columns, start_index, end_index):
    """PLot the desired columns over index values in the given range."""
    # Filter columns
    df = df[columns]
    
    # Slice by row range
    df = df[start_index:end_index]
    
    # Plot stock data
    plt.figure(figsize=(10,5), dpi=200)
    plt.plot(df)
    plt.grid(color='blue', linestyle='--', linewidth=1, alpha=0.2)
    plt.title("Stock prices")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend(df)
    plt.show()
    
    
def test_run():
    # Define a data range
    dates = pd.date_range('2005-01-01', '2012-12-31') # year 2010
    
    # Choose stock symbols to read
    symbols = ['SPY', 'JAVA']  # SPY will be added in get_data()
    
    # Get stock data
    df = get_data(symbols=symbols, dates=dates)
    
    # Slice and plot
    plot = plot_selected(df, symbols, '2005-01-01', '2012-12-31')
    return plot

test_run()

### Why this is bad - what can we do?
1. Fill forward from last previous known value.
2. Can't use interpolation since we would be giving information about the future (not allowed).
3. Fill backward after filling forward.

### Pandas fillna()
Fill forward missing values.
`df.fillna(method="ffill")`

In [ ]:
def plot_data(df, title='Stock prices', xlabel='Date', ylabel='Price'):
    plt.figure(figsize=(10,5), dpi=200)
    plt.plot(df)
    plt.grid(color='blue', linestyle='--', linewidth=1, alpha=0.2)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(df)
    plt.show()


def test_run():
    # Define a data range
    start, end = '2005-12-31', '2014-12-07'
    dates = pd.date_range(start, end) # year 2010
    
    # Choose stock symbols to read
    symbols = ['SPY', 'FAKE2']  # SPY will be added in get_data()
    
    # Get stock data
    df = get_data(symbols=symbols, dates=dates)
    df.fillna(method="ffill", inplace=True)
    
    # Plot data
    plot = plot_data(df, title='Incomplete Data')
    return plot

test_run()

### Fill missing values

In [ ]:
def test_run():
    # Define a data range
    start, end = '2005-12-31', '2014-12-07'
    dates = pd.date_range(start, end) # year 2010
    
    # Choose stock symbols to read
    symbols = ['JAVA', 'FAKE1', 'FAKE2']  # SPY will be added in get_data()
    
    # Get stock data
    df = get_data(symbols=symbols, dates=dates)
    df.fillna(method="ffill", inplace=True)
    df.fillna(method='bfill', inplace=True)
    
    # Slice and plot
    plot = plot_data(df)
    return plot

test_run()

## 01-06 Histograms and scatterplots

### Histogram of daily returns
- **Standard deviation:** on average, how far the individual values **deviate** from the mean.
- **Mean:** average value
- **Kurtosis:**
    - Fat tail (+)
    - Skinny tail (-)

### How to plot a histogram

In [ ]:
def compute_daily_returns(df):
    """Compute and return the daily return values."""
    daily_returns = df.copy()
    daily_returns[1:] = (df.iloc[1:] / df.iloc[:-1].values) - 1
    daily_returns.iloc[0, :] = 0
    return daily_returns

def plot_hist(df, bins=10, title='Histogram', xlabel='Returns', ylabel='Frequency'):
    plt.figure(figsize=(10,5), dpi=200)
    plt.hist(df, bins=bins, edgecolor='black')
    plt.grid(color='blue', linestyle='--', linewidth=1, alpha=0.2)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(df)
    plt.show()
    

def test_run():
    # Define a data range
    start, end = '2009-01-01', '2012-12-31'
    dates = pd.date_range(start, end) # year 2010
    
    # Choose stock symbols to read
    symbols = ['SPY']  # SPY will be added in get_data()
    
    # Get stock data
    df = get_data(symbols=symbols, dates=dates)
    
    # Plot data
    plot_data(df, title='Stock price')
    
    # Compute daily returns
    daily_returns = compute_daily_returns(df)
    plot_data(daily_returns, title='Daily returns', xlabel='Date', ylabel='Daily returns')
    
    # Plot a histogram
    plot_hist(daily_returns, bins=20, xlabel='Returns', ylabel='Frequency') # default bins = 10
    
test_run()

### Computing histogram statistics

In [ ]:
def plot_hist_mean_std(df, mean, std, bins=10, title='Histogram', xlabel='Returns', ylabel='Frequency'):
    plt.figure(figsize=(10,5), dpi=200)
    plt.hist(df, bins=bins, edgecolor='black')
    plt.axvline(mean, color='w', linestyle='dashed', linewidth=2)
    plt.axvline(std, color='r', linestyle='dashed', linewidth=2)
    plt.axvline(-std, color='r', linestyle='dashed', linewidth=2)
    plt.grid(color='blue', linestyle='--', linewidth=1, alpha=0.2)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(df)
    plt.show()


def test_run():
    # Define a data range
    start, end = '2009-01-01', '2012-12-31'
    dates = pd.date_range(start, end) # year 2010
    
    # Choose stock symbols to read
    symbols = ['SPY']  # SPY will be added in get_data()
    
    # Get stock data
    df = get_data(symbols=symbols, dates=dates)
    
    # Plot data
    # plot_data(df, title='Stock price')
    
    # Compute daily returns
    daily_returns = compute_daily_returns(df)
    # plot_data(daily_returns, title='Daily returns', xlabel='Date', ylabel='Daily returns')
    
    
    # Get mean and standard deviation
    mean = daily_returns['SPY'].mean()
    print("Mean", np.round(mean, 3))
    std = daily_returns['SPY'].std()
    print("Standard Deviation:", np.round(std, 3))
    # Compute kurtosis
    print("Kurtosis:", daily_returns.kurtosis())
    
    # Plot a histogram
    plot = plot_hist_mean_std(daily_returns, mean=mean, std=std, bins=20, 
                              title='Histogram', xlabel='Returns', ylabel='Frequency') # default bins = 10
    
    
test_run()

### Plot two histograms together

In [ ]:
def test_run():
    # Define a data range
    start, end = '2009-01-01', '2012-12-31'
    dates = pd.date_range(start, end) # year 2010
    
    # Choose stock symbols to read
    symbols = ['SPY', 'XOM']  # SPY will be added in get_data()
    
    # Get stock data
    df = get_data(symbols=symbols, dates=dates)
    
    # Compute daily returns
    daily_returns = compute_daily_returns(df)
    
    # Plot a histogram
    plot_hist(daily_returns, bins=20, title='Histogram', xlabel='Returns', ylabel='Frequency') # default bins = 10

    
test_run()

### Slope does not equal correlation
Correlation is a measure of how tightly the values fit the line.

### Scatterplots in python

In [ ]:
def plot_scatter(df, x, y, title='Stock prices', xlabel='Date', ylabel='Price'):
    plt.figure(figsize=(10,5), dpi=200)
    
    # Fit a line
    beta, alpha = np.polyfit(df[x], df[y], 1)
    plt.plot(df[x], beta*df[x] + alpha, color='r', linewidth=2.5)
    
    plt.scatter(x=df[x], y=df[y], edgecolor='black', s=60, alpha=0.7)
    plt.grid(color='blue', linestyle='--', linewidth=1, alpha=0.2)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend([f"Beta = {np.round(beta,3)}\nAlpha = {np.round(alpha,3)}"])
    plt.show()
    

def test_run():
    # Define a data range
    start, end = '2009-01-01', '2012-12-31'
    dates = pd.date_range(start, end) # year 2010
    
    # Choose stock symbols to read
    symbols = ['SPY', 'XOM', 'GLD']  # SPY will be added in get_data()
    
    # Get stock data
    df = get_data(symbols=symbols, dates=dates)
    
    # Compute daily returns
    daily_returns = compute_daily_returns(df)
    
    # Scatterplot SPY vs XOM
    plot_scatter(daily_returns, x='SPY', y='XOM', title='SPY vs XOM Scatterplot', 
                 xlabel='SPY', ylabel='XOM')
    
    # Scatterplot SPY vs GLD
    plot_scatter(daily_returns, x='SPY', y='GLD', title='SPY vs GLD Scatterplot', 
                 xlabel='SPY', ylabel='GLD')
    
    # Compute correlation coefficient
    corr = daily_returns.corr(method='pearson')
    print(f"Correlation:\n{corr}")
    
test_run()

In [ ]:
def plot_data(df, title='Stock prices'):
    """Plot stock prices"""
    plt.figure(figsize=(10,5), dpi=200)
    plt.plot(df)
    plt.grid(color='blue', linestyle='--', linewidth=1, alpha=0.2)
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend(df)
    plt.show()
    
    
def test_run():
    # Define a data range
    start, end = '2009-01-01', '2012-12-31'
    dates = pd.date_range(start, end) # year 2010
    
    # Choose stock symbols to read
    symbols = ['SPY', 'XOM', 'GLD']  # SPY will be added in get_data()
    
    # Get stock data
    df = get_data(symbols=symbols, dates=dates)
    
    # Plot data
    plot_data(df)

    
test_run()

## 01-07 Sharpe ratio and other portfolio statistics

### Sharpe ratio and other portfolio statistics overview
Portfolio: allocation of funds to a set of stocks.


### Daily portfolio values
1. Prices dataframe with stocks in our portfolio.
2. Normalize data: `normed = prices/prices[0]`
3. Allocate weights for each stock in portfolio and multiply them.
4. Pos-vals: multiply allocated dataframe with start value.
5. Port-val: calculate the sum value of each day for the row (axis=1).

In [ ]:
# Computing daily returns
# Define a data range
start, end = '2009-01-01', '2012-12-31'
dates = pd.date_range(start, end) # year 2010

# Choose stock symbols to read
symbols = ['SPY', 'XOM', 'GLD']  # SPY will be added in get_data()

# Get stock data
df = get_data(symbols=symbols, dates=dates)

daily_returns = df.copy()
daily_returns[1:] = (df.iloc[1:] / df.iloc[:-1].values) - 1
daily_returns.iloc[0, :] = 0

In [ ]:
daily_returns.head()

### Portfolio statistics
- **Cumulative return:** how much the value of the portfolio has gone up from the beginning to the end.
- **Avg. daily return:** is the average of the daily return
- **Std. daily return:** is the std. of the daily return
- **Sharpe ratio:** risk adjusted return. Lower risk/higher return is better. Also considers risk free rate of return.

### Form of the Sharpe ratio

$\large
 \begin{array}{l}
S_{a} =\frac{E[ R_{a} -R_{b}]}{\sigma _{a}}\\
\\
S_{a} =\text{Sharpe ratio}\\
E=\text{expected value}\\
R_{a} =\text{asset return}\\
R_{b} =\text{risk free return}\\
\sigma _{a} =\text{standard deviation of the asset excess return}
\end{array}
$

### Computing Sharpe ratio
$\large
 \begin{array}{l}
S=\frac{\text{mean(daily_returns } -\text{ daily_risk_free)}}{\text{std(daily_returns } -\text{ daily_risk_free)}}\\
\\
S=\frac{\text{mean(daily_returns } -\text{ daily_rf)}}{\text{std(daily_returns})}
\end{array}
$

What is the **risk free rate**?
- LIBOR
- 3-month treasury bill
- 0%

$\large
\text{daily_rf} =\sqrt[252]{1.0+0} -1
$

### But wait, there's more!
- SR can vary widely depending on how frequently you sample
- SR is an annual measure
- $SR\ \text{annualized} =K\times SR$
- $K=\sqrt{\text{# samples per year}}$
- $ \begin{array}{l}
\text{daily K} =\sqrt{252}\\
\text{weekly K} =\sqrt{52}\\
\text{monthly K} =\sqrt{12}
\end{array}$

Sharpe ratio summary:

$\large
SR=\sqrt{252} \times \frac{\text{mean(daily_returns } -\text{ daily_rf)}}{\text{std(daily_returns})}
$

### Quiz: What is the Sharpe ratio?
- 60 days of data
- avg. daily ret = 10 bps = 0.001
- daily risk free = 2 bps = 0.0002
- std. daily ret = 10 bps = 0.001

In [ ]:
daily_ret = 0.001
daily_rf = 0.0002
std_dr =  0.001

# Compute Sharpe ratio
SR = np.sqrt(252)*(np.mean(daily_ret - daily_rf)/std_dr)
print(f'Sharpe ratio = {np.round(SR, 3)}')

## 01-08 Optimizers: Building a parameterized model

## What is an optimizer?
#### Optimizers
- Find minimum values of functions.
- Build parameterized models based on data.
- Refine allocations to stocks in portfolios (e.g., what percentage of funds should be allocated to each stock).

How to use an optimizer?
1. Provide a function to minimize.
2. Provide an initial guess.
3. Call the optimizer.

In [ ]:
import scipy.optimize as spo

### Minimizer in Python
Minimize an objective function, using SciPy.

In [ ]:
def f(x):
    """Given a scalar X, return some value (a real number)."""
    y = (x -1.5)**2 + 0.5
    print(f"x = {x}, y = {y}")
    return y

def test_run():
    Xguess = 2.0
    min_result = spo.minimize(f, Xguess, method='SLSQP', options={'disp': True})
    print(f"Minima found at: X = {min_result.x}, y = {min_result.fun}")
    
    # Plot function values, mark minima
    Xplot = np.linspace(0.5, 2.5, 21)
    Yplot = f(Xplot)
    plt.figure(figsize=(10,6), dpi=200)
    plt.grid(color='blue', linestyle='--', linewidth=1, alpha=0.2)
    plt.plot(Xplot, Yplot)
    plt.plot(min_result.x, min_result.fun, 'ro')
    plt.title("Minima of an objective function")
    plt.show()
    
test_run()

### Convex problems
A real valued function $f(x)$ defined on an interval is called **convex** is the line segment between any two points in the function lies above the graph.
- Convex function has only one **local minima** (where local minima = global minima).
- No flat regions.

### Building a parameterized model
$f(x) = w\cdot x + b$

Objective: find the optimal value of $w$ that minimizes the cost of the function.

In [ ]:
def error(line, data):  # error function
    """Compute error between given line model and observed data.

    :param line: tuple, list, array (w, b) where w is slope and b is Y-intercept
    :param data: 2D array where each row is a point (x, y)
    :return: error as a single real value.
    """
    # Metric: Sum of squared Y-axis differences
    err = np.sum((data[:,1] - (line[0] * data[:, 0] + line[1])) ** 2)
    return err	


def fit_line(data, error_func):
    """Fit a line to given data, using a supplied error function.

    :param data: 2D array where each row is a point (x, y).
    :param error_func: function that computes the error between al ine and observed data.
    :return: line that minimizes the error function.
    """
    # Generate initial guess for line model 
    l = np.float32([0, np.mean(data[:, 1])])  # slope = 0, intercept = mean(y values)

    # Plot initial guess (optional)
    x_ends = np.float32([-5, 5])
    plt.plot(x_ends, l[0] * x_ends + l[1], 'm--', linewidth=2.0, label = "Initial guess")

    # Call optimizer to minimize error function
    result = spo.minimize(error_func, l, args=(data,), method = 'SLSQP', options={'disp': True})
    return result.x


def test_run():
    # Define original line
    l_orig = np.float32([4, 2])
    print(f"Original line: w = {l_orig[0]}, b = {l_orig[1]}")
    Xorig = np.linspace(0, 10, 21)
    Yorig = l_orig[0] * Xorig + l_orig[1]
    plt.figure(figsize=(10,6), dpi=200)
    plt.grid(color='blue', linestyle='--', linewidth=1, alpha=0.2)
    plt.plot(Xorig, Yorig, 'b--', linewidth=2.0, label='Original line')

    # Generate noisy data points
    noise_sigma = 3.0
    noise = np.random.normal(0, noise_sigma, Yorig.shape)
    data = np.asarray([Xorig, Yorig + noise]).T
    plt.plot(data[:, 0], data[:, 1], 'go', label='Data points')

    # Try to fit a line to this data
    l_fit = fit_line(data, error)
    print(f"Fitted line: w = {l_fit[0]}, b = {l_fit[1]}")
    plt.plot(data[:, 0], l_fit[0] * data[:, 0] + l_fit[1], 'r--', linewidth=2.0, label='Data points')

    # Add a legend and show plot
    plt.legend(loc='upper left')
    plt.show()

    
test_run()

### And it works for polynomials too!

In [ ]:
def error_poly(C, data): # error function
    """Compute error between given polynomial and observed data.

    Parameters
    ----------
    C: numpy.poly1d object or equivalent array representing polynomial coefficients
    data: 2D array where each row is a point (x, y)

    Returns error as a single real value.
    """
    # Metric: Sum of squared Y-axis differences
    err = np.sum((data[:,1] - np.polyval(C, data[:,0])) ** 2)
    return err


def fit_poly(data, error_func, degree=3):
    """Fit a polynomial to given data, using a supplied error function.

    Parameters
    ----------
    data: 2D array where each row is a point (X0, Y)
    error_func: function that computes the error between a polynomial and observed data

    Returns polynomial that minimizes the error function.
    """
    # Generate initial guess for line model (all coeffs = 1)
    Cguess = np.poly1d(np.ones(degree + 1, dtype=np.float32))

    # Plot initial guess (optional)
    x = np.linspace(-5, 5, 21)
    plt.plot(x, np.polyval(Cguess, x), 'm--', linewidth=2.0, label = "Initial guess")

    # Call optimizer to minimize error function
    result = spo.minimize(error_func, Cguess, args=(data,), method = 'SLSQP', options={'disp': True})
    return np.poly1d(result.x) # convert optimal result into a poly1d object and return


def test_run():
    # Define original line

    # Generate noisy data points


    # Try to fit a line to this data


    # Add a legend and show plot
    return


test_run()